# Zip Code with Gage Figures
_Calvin Whealton_

This notebook loops through all the zip codes that are in the contiguous 48 states with a zip code tabulation area. The gages for the zip code are plotted based on the distance and the trend in the time series. These plots represent whether the floods in a location are getting larger or smaller on average with time.

In [103]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib as mpl
import os
from shapely.geometry import box
from shapely.geometry import Polygon
import rtree

### River Shapefile (Reading in and Reprojecting)

In [2]:
# loading in the rivers and streams shapefile
# file downloaded from https://hub.arcgis.com/datasets/esri::usa-rivers-and-streams?geometry=57.222%2C-0.831%2C64.605%2C76.487
# also uploaded at https://drive.google.com/drive/folders/1LjXPNTfeesGjYOHW1HmZ6x2eDTmg_6oy?usp=sharing
os.chdir('/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/data/geo_data/USA_Rivers_and_Streams-shp')
rivers = gpd.read_file('9ae73184-d43c-4ab8-940a-c8687f61952f2020328-1-r9gw71.0odx9.shp')

In [7]:
# this confirms that the coordinate reference system (CRS) is WGS84 (decimal lat and long)
# will reproject to a distance-based projection
rivers.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
# projecting to EPSG:2163
# US National Atlas projection
rivers_dist = rivers.to_crs('EPSG:2163')

### Zip Code Tabulation Area Shapefile (Reading in and Reprojecting)

In [9]:
# loading in the zip code tabulation area (ZCTA) shapefile
# available as a Tigerline shapefile from the US Census
# also uploaded to https://drive.google.com/drive/folders/1z3JkCNWx-PuLXD_cuMLPa72Xcuk7lyI3?usp=sharing
os.chdir('/Users/calvinwhealton/Documents/GitHub/tdi_capstone/data/geo_data/tl_2019_us_zcta510_clipped48contig')
zctas = gpd.read_file('clipped48contig.shp')

In [10]:
# this confirms that the coordinate reference system (CRS) is WGS84 (decimal lat and long)
# will reproject to a distance-based projection
zctas.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - NAD83
- bounds: (167.65, 14.92, -47.74, 86.46)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [11]:
# projecting to EPSG:2163
# US National Atlas projection
zctas_dist = zctas.to_crs('EPSG:2163')

In [46]:
zctas_dist.head()

,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,GEOID10_str
0,43451,43451,B5,G6350,S,63484186,157689,+41.3183010,-083.6174935,"POLYGON ((1350004.658 -273873.705, 1350052.741...",43451
1,43452,43452,B5,G6350,S,121522304,13721730,+41.5157923,-082.9809454,"POLYGON ((1395993.519 -240385.238, 1396351.828...",43452
2,43456,43456,B5,G6350,S,9320975,1003775,+41.6318300,-082.8393923,"MULTIPOLYGON (((1412486.411 -217125.781, 14125...",43456
3,43457,43457,B5,G6350,S,48004681,0,+41.2673301,-083.4274872,"POLYGON ((1368878.803 -278420.738, 1369913.597...",43457
4,43458,43458,B5,G6350,S,2573816,39915,+41.5304461,-083.2133648,"POLYGON ((1385168.510 -243426.788, 1385134.274...",43458


### Stream Gages (Reading in and Reprojecting)

In [13]:
# loading in the stream gage information
# downloaded as a text file from USGS website
os.chdir('/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/data/gage_data')
gages = pd.read_csv('usgs_supp.txt',sep='\t',comment='#')

In [130]:
gages.head()

,agency_cd,site_no,station_nm,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,state_cd,county_cd,alt_va,alt_acy_va,alt_datum_cd,basin_cd,contrib_drain_area_va,geometry
0,USGS,1010000,"St. John River at Ninemile Bridge, Maine",46.700556,-69.715556,S,NAD83,23.0,3.0,931.26,0.01,NGVD29,NaN,1341.0,POINT (-69.71556 46.70056)
1,USGS,1010070,"Big Black River near Depot Mtn, Maine",46.893889,-69.751667,S,NAD83,23.0,3.0,885.00,20.00,NGVD29,NaN,171.0,POINT (-69.75167 46.89389)
2,USGS,1010500,"St. John River at Dickey, Maine",47.113056,-69.088056,S,NAD83,23.0,3.0,590.38,0.01,NGVD29,NaN,2680.0,POINT (-69.08806 47.11306)
3,USGS,1011000,"Allagash River near Allagash, Maine",47.069722,-69.079444,S,NAD83,23.0,3.0,604.60,0.01,NGVD29,NaN,1229.0,POINT (-69.07944 47.06972)
4,USGS,1013500,"Fish River near Fort Kent, Maine",47.237500,-68.582778,S,NAD83,23.0,3.0,511.38,0.01,NGVD29,NaN,873.0,POINT (-68.58278 47.23750)


In [133]:
min(gages['contrib_drain_area_va'].values), max(gages['contrib_drain_area_va'].values)

(0.0, 1140500.0)

In [20]:
# dropping locations without coordinates
gages.dropna(subset=['dec_lat_va','dec_long_va'],inplace=True)
gages_gdf = gpd.GeoDataFrame(gages,geometry=gpd.points_from_xy(gages.dec_long_va, gages.dec_lat_va))
gages_gdf = gpd.GeoDataFrame(gages,columns=['site_no','geometry'])

In [21]:
gages_gdf.crs = {'init' :"EPSG:4269"}
gages_gdf_dist = gages_gdf.to_crs('EPSG:2163')
gages_gdf_dist['x'] = gages_gdf_dist['geometry'].x
gages_gdf_dist['y'] = gages_gdf_dist['geometry'].y

/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [23]:
gages_gdf_dist.head()

,site_no,geometry,x,y
0,1010000,POINT (2241049.472 621122.518),2.241049e+06,621122.517752
1,1010070,POINT (2230437.221 640396.859),2.230437e+06,640396.858568
2,1010500,POINT (2266857.514 682270.118),2.266858e+06,682270.117604
3,1011000,POINT (2269310.429 677982.898),2.269310e+06,677982.898332
4,1013500,POINT (2296028.248 709900.183),2.296028e+06,709900.183249


### Gages for Zip Code (Reading-in file)

In [24]:
# reading-in the file that includes the closest gages to each zip code
os.chdir('/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/data/processed_data')
gage_zip = pd.read_csv('zip_gage_dist_2020-08-10.csv')

In [53]:
gage_zip['GEOID10_str'] = gage_zip['GEOID10'].astype(str).str.pad(width=5, side='left', fillchar='0')

In [54]:
gage_zip.head()

,Unnamed: 0,ZCTA5CE10,GEOID10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,x,y,geometry,...,dist5,gage6,dist6,gage7,dist7,gage8,dist8,gage9,dist9,GEOID10_str
0,0,43451,43451,63484186,157689,41.318301,-83.617494,-83.617494,41.318301,POINT (1357632.981903858 -273355.0236152691),...,35048.269737,4195820,38047.461696,4177000,38127.199412,4198000,38153.116744,4176900,38912.761534,43451
1,1,43452,43452,121522304,13721730,41.515792,-82.980945,-82.980945,41.515792,POINT (1405077.788522715 -240979.1349070556),...,38992.381755,4199155,42889.448497,4197170,47669.895781,4198100,48099.579127,4177000,54583.146601,43452
2,2,43456,43456,9320975,1003775,41.631830,-82.839392,-82.839392,41.631830,POINT (1413951.1585467 -225837.3870959734),...,48263.019518,4198100,50798.462462,4199500,51313.907210,4197300,54115.851576,4176605,56928.044679,43456
3,3,43457,43457,48004681,0,41.267330,-83.427487,-83.427487,41.267330,POINT (1374212.851300819 -275772.6169586082),...,30266.616017,4197000,31432.801554,4189000,32218.889964,4192900,34624.114203,4193500,35022.435948,43457
4,4,43458,43458,2573816,39915,41.530446,-83.213365,-83.213365,41.530446,POINT (1385905.375784243 -243339.0096639248),...,35956.931241,4176900,40417.635485,4192900,41341.050532,4193500,41563.461857,4176605,42369.956528,43458


In [122]:
# reading-in the file for trends in the gages
# information used in plotting the maps
os.chdir('/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/data/processed_data')
gage_trends = pd.read_csv('gage_trends.csv')

### Making Maps

In [33]:
zctas_dist['GEOID10_str'] = zctas_dist['GEOID10'].str.pad(width=5, side='left', fillchar='0')

In [50]:
gage_cols_nms = ['gage0','gage1','gage2','gage3','gage4','gage5','gage6','gage7','gage8','gage9']



In [60]:
np.ndarray.flatten((gage_zip.loc[gage_zip['GEOID10_str']==gage_zip['GEOID10_str'].values[10],gage_cols_nms]).values)

array([4195500, 4197300, 4197500, 4192900, 4193500, 4198000, 4189000,
       4195820, 4197170, 4197000])

In [237]:
import matplotlib.pyplot as plt

# column names to extract the gage numbers for each zip code
gage_cols_nms = ['gage0','gage1','gage2','gage3','gage4','gage5','gage6','gage7','gage8','gage9']
dist_cols_nms = ['dist0','dist1','dist2','dist3','dist4','dist5','dist6','dist7','dist8','dist9']

size=25
params = {'legend.fontsize': 'large',
          'figure.figsize': (20,8),
          'axes.labelsize': size,
          'axes.titlesize': size,
          'xtick.labelsize': size*0.75,
          'ytick.labelsize': size*0.75,
          'axes.titlepad': 25}

plt.rcParams.update(params)

os.chdir('/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/visualizations/zip_results/zip_gages')

for z in zctas_dist['GEOID10_str'].values:
    
    # extract the gage numbers and distances to zip code and flatten it to a list
    gages_for_zip = np.ndarray.flatten((gage_zip.loc[gage_zip['GEOID10_str']==z,gage_cols_nms]).values)
    dist_for_zip = np.ndarray.flatten((gage_zip.loc[gage_zip['GEOID10_str']==z,dist_cols_nms]).values)
    
    # dataframe for gages
    gage_temp = pd.DataFrame()
    for g in gages_for_zip:
        gage_temp = gage_temp.append(gage_trends.loc[gage_trends['gage']==g])    
    
    gage_temp.reset_index(inplace=True)
    
    gage_temp['dist'] = 0
    for ind in gage_temp.index:
        dister = dist_for_zip[gages_for_zip==gage_temp.loc[ind,'gage']]
        if len(dister) == 1:
            gage_temp.loc[ind,'dist'] = dister
        else:
            gage_temp.loc[ind,'dist'] = dister[0]
    
    # making the figure
    plt.figure(figsize=(10,8))
    plt.scatter(gage_temp['dist'].values/1000,abs(gage_temp['tau'].values),
                s=100*abs(np.log10(abs(gage_temp['slope_rel_ref'].values))),
                linewidths=3,
                c=symbol_fill(gage_temp),
                edgecolors=symbol_color(gage_temp)
               )
    plt.title('Closest ' + str(gage_temp.shape[0]) + ' Gages for zip code '+ z)
    plt.xlabel('Distance (km)')
    plt.ylabel('Time Trend (Kendall tau)')
    plt.savefig(z + '_zip_gage'+'.png')
    plt.close()
    
    

In [140]:
def symbol_fill(gage_temp,increase='#fdae61',decrease='#abd9e9',ptest=0.1):
    '''
    function for whether the plotting symbol will be full or empty
    full indicates a statistically significant result
    '''
    
    fillers = []
    
    for ind in gage_temp.index:
        if gage_temp.loc[ind,'pvalue'] <= ptest:
            if gage_temp.loc[ind,'tau'] > 0:
                fillers.append(increase)
            else:
                fillers.append(decrease)
        else:
            fillers.append('white')
            
    return fillers

In [173]:
def symbol_color(gage_temp,increase='#fdae61',decrease='#abd9e9'):
    '''
    function to assign the colors for the points
    points with positive trend are assigned increase, below zero are decrease
    '''
    cols = []
    for ind in gage_temp.index:
        if gage_temp.loc[ind,'tau'] < 0:
            cols.append(decrease)
        else:
            cols.append(increase)
            
    return cols

In [112]:
# from shapely.geometry import MultiPoint
# from shapely import geometry

# def find_bounds(zipcode,gages,zctas):
#     '''
#     function to compute the bounds for maps for each zip code
#     box needs to include the whole zip code area and the closest stream gages
#     zipcode is a 5-digit string
#     gages is a list of gages for the area
#     zctas is a shapefile of the zip code (zip code tabulation area)
    
#     returns a shapely polygon of the bounding box
#     '''
    
#     # calculating bounds for the zip code polygon
#     z_shape = zctas_dist.loc[zctas_dist['GEOID10_str']==zipcode]
#     zip_bounds = z_shape.bounds
    
#     # calculating bounds for the gages
#     gage_y_min = min(gages['y'].values)
#     gage_y_max = max(gages['y'].values)
    
#     gage_x_min = min(gages['x'].values)
#     gage_x_max = max(gages['x'].values)
    
#     # bounding box dimension
#     # [xmin,xmax,ymin,ymax]
#     bounds = np.array([min(gage_x_min,zip_bounds['minx'].values[0]),
#             max(gage_x_max,zip_bounds['maxx'].values[0]),
#             min(gage_y_min,zip_bounds['miny'].values[0]),
#             max(gage_y_max,zip_bounds['maxy'].values[0])
#            ])
    
#     # this balances the box to make it square
#     if (bounds[1] - bounds[0]) > (bounds[3]-bounds[2]):
#         bounds[2] = bounds[2] - 0.5*((bounds[1] - bounds[0]) - (bounds[3]-bounds[2]))
#         bounds[3] = bounds[3] + 0.5*((bounds[1] - bounds[0]) - (bounds[3]-bounds[2]))
#     else:
#         bounds[0] = bounds[0] + 0.5*((bounds[1] - bounds[0]) - (bounds[3]-bounds[2]))
#         bounds[1] = bounds[1] - 0.5*((bounds[1] - bounds[0]) - (bounds[3]-bounds[2]))
    
#     bounds = np.ndarray.flatten(bounds)
    
#     # make a list
#     x_v = [bounds[0], bounds[0], bounds[1], bounds[1]]
#     y_v = [bounds[2], bounds[3], bounds[3], bounds[2]]
    
#     # make points
#     p1 = geometry.Point(x_v[0],y_v[0])
#     p2 = geometry.Point(x_v[1],y_v[1])
#     p3 = geometry.Point(x_v[2],y_v[2])
#     p4 = geometry.Point(x_v[3],y_v[3])
    
#     # list of points
#     pointList = [p1, p2, p3, p4, p1]
    
#     # making a shapely polygon
#     bounds_poly = geometry.Polygon([[p.x, p.y] for p in pointList])

#     return bounds_poly
    

In [ ]:
# def map_zip_gage(zipcode,rivers,gages,zcta):
#     '''
#     funtion to generate map with gages and rivers for a zip code
#     zipcode is a 5-digit string
#     rivers is a shapefile of the rivers trimmed to the relevant area
#     gages is a list of gages for the area
#     zctas is a shapefile of the zip code tabulation area
#     '''
    
    